In [ ]:
current_folder = getwd()

require(data.table)
require(lubridate)

dat = fread('YAT-01012019-10122021.csv')

setnames(dat,names(dat),c('date','hour','yat_one','yat_two','yat_three'))
dat[,datex:=strptime(date,'%d/%m/%Y')]

dat[,tst:=ymd_hm(paste(datex,hour))]
dat[,date:=date(tst)]
dat[,hour:=hour(tst)]

dat[,yat_one_t:=gsub('\\.','',yat_one)]
dat[,yat_two_t:=gsub('\\.','',yat_two)]
dat[,yat_three_t:=gsub('\\.','',yat_three)]


dat[,yat_one:=as.numeric(gsub(',','.',yat_one_t))]
dat[,yat_two:=as.numeric(gsub(',','.',yat_two_t))]
dat[,yat_three:=as.numeric(gsub(',','.',yat_three_t))]


yat_dat = dat[,list(date,hour,yat_one,yat_two,yat_three)]


dat=fread('YAL-01012019-10122021.csv')

# naming is temporary here, used the same set of codes
setnames(dat,names(dat),c('date','hour','yat_one','yat_two','yat_three'))
dat[,datex:=strptime(date,'%d/%m/%Y')]

dat[,tst:=ymd_hm(paste(datex,hour))]
dat[,date:=date(tst)]
dat[,hour:=hour(tst)]

dat[,yat_one_t:=gsub('\\.','',yat_one)]
dat[,yat_two_t:=gsub('\\.','',yat_two)]
dat[,yat_three_t:=gsub('\\.','',yat_three)]


dat[,yal_one:=as.numeric(gsub(',','.',yat_one_t))]
dat[,yal_two:=as.numeric(gsub(',','.',yat_two_t))]
dat[,yal_three:=as.numeric(gsub(',','.',yat_three_t))]


yal_dat = dat[,list(date,hour,yal_one,yal_two,yal_three)]

In [55]:
# funtion to preprocess the data 
preprocess = function(yat_data, yal_data, buffer_zone_start, window_size){
    
    dat = merge(yat_dat,
                yal_dat,
                by=c("date","hour"),
                all=FALSE)

    dat = dat[, list(yat_vol = yat_one + yat_two + yat_three,
                           yal_vol = yal_one + yal_two + yal_three), by=c("date", "hour")]
    dat[, net_imb:=yat_vol-yal_vol]
    
    # eliminate edge cases found by the distribition
    dat[, net_imb:=ifelse(net_imb<(-5000), (-5000), ifelse(net_imb>5000, 5000, net_imb))]

    # add clasifier column
    dat[, direction:=ifelse(net_imb>50, "surplus", ifelse(net_imb<(-50),"deficit", "balanced"))]
    
    # reorder column for easiness
    dat = dat[,c(1,3,4,5,6,2)]
    setnames(dat, "hour", "target")
    
    # lag the hours according to window_size to create a matrix
    preprocessed_data = dat[, paste0((buffer_zone_start+1):(buffer_zone_start+window_size), "_hours_before") := shift(net_imb, (buffer_zone_start+1):(buffer_zone_start+window_size))]
    
    # remove the rows having NA lags values due to time boundry of the data and select only the complete_cases
    preprocessed_data = preprocessed_data[complete.cases(preprocessed_data), ]
    
    # remove columns that won't be used
    processed_data = preprocessed_data[,-c("yat_vol", "yal_vol")]
    
    return(processed_data)
}

In [58]:
different_windows_data = list()
data_window_size = c(100,200)
iter=1
for (window in data_window_size){
    processed_data = preprocess(yat_dat, yal_dat, buffer_zone_start=buffer_zone_start, window_size=window)
    different_windows_data[[iter]] = processed_data
    iter=iter+1
}

In [59]:
different_windows_data

date,net_imb,direction,target,1_hours_before,2_hours_before,3_hours_before,4_hours_before,5_hours_before,6_hours_before,...,91_hours_before,92_hours_before,93_hours_before,94_hours_before,95_hours_before,96_hours_before,97_hours_before,98_hours_before,99_hours_before,100_hours_before
2019-01-05,1327.00,surplus,4,1523.00,1521.00,1503.84,2183.42,2252.33,2009.08,...,2614.72,2509.77,1963.00,2086.00,2181.00,1930.63,1120.48,1060.75,808.08,2288.57
2019-01-05,1609.00,surplus,5,1327.00,1523.00,1521.00,1503.84,2183.42,2252.33,...,2417.95,2614.72,2509.77,1963.00,2086.00,2181.00,1930.63,1120.48,1060.75,808.08
2019-01-05,1908.00,surplus,6,1609.00,1327.00,1523.00,1521.00,1503.84,2183.42,...,1927.40,2417.95,2614.72,2509.77,1963.00,2086.00,2181.00,1930.63,1120.48,1060.75
2019-01-05,1846.93,surplus,7,1908.00,1609.00,1327.00,1523.00,1521.00,1503.84,...,944.00,1927.40,2417.95,2614.72,2509.77,1963.00,2086.00,2181.00,1930.63,1120.48
2019-01-05,1815.51,surplus,8,1846.93,1908.00,1609.00,1327.00,1523.00,1521.00,...,81.97,944.00,1927.40,2417.95,2614.72,2509.77,1963.00,2086.00,2181.00,1930.63
2019-01-05,2098.06,surplus,9,1815.51,1846.93,1908.00,1609.00,1327.00,1523.00,...,-277.87,81.97,944.00,1927.40,2417.95,2614.72,2509.77,1963.00,2086.00,2181.00
2019-01-05,1662.57,surplus,10,2098.06,1815.51,1846.93,1908.00,1609.00,1327.00,...,-227.68,-277.87,81.97,944.00,1927.40,2417.95,2614.72,2509.77,1963.00,2086.00
2019-01-05,710.00,surplus,11,1662.57,2098.06,1815.51,1846.93,1908.00,1609.00,...,485.57,-227.68,-277.87,81.97,944.00,1927.40,2417.95,2614.72,2509.77,1963.00
2019-01-05,372.22,surplus,12,710.00,1662.57,2098.06,1815.51,1846.93,1908.00,...,1215.90,485.57,-227.68,-277.87,81.97,944.00,1927.40,2417.95,2614.72,2509.77
2019-01-05,118.75,surplus,13,372.22,710.00,1662.57,2098.06,1815.51,1846.93,...,1228.83,1215.90,485.57,-227.68,-277.87,81.97,944.00,1927.40,2417.95,2614.72
